In [2]:
# Libraries
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 16)
#pd.set_option('display.width', 2000)
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [4]:
# Load potential antibiotics 
infection_treatment_categories = pd.read_csv(r"eicu-collaborative-research-database-2.0/infection_treatment_categories.csv")
infection_treatment_categories2 = infection_treatment_categories.rx.str.split(pat='|', expand=True)
infection_treatment_categories2.rename(columns={0:'a', 1:'b', 2:'c', 3:'d', 4:'e'}, inplace=True)
infection_treatment_categories3 = infection_treatment_categories2.d.str.split(pat='/', expand=True)
# Create lists of antibiotics 
list1 = infection_treatment_categories2.d.to_list()
list2 = infection_treatment_categories2.e.to_list()
# Add: 'linezolid', 'Zyvox', 'Synercid', 'quinupristin', 'dalfopristin', 'quinupristin/dalfopristin', 'cephalosporin', 'ticarcillin', 'amoxicillin', 'penicillin', 'benzathine', 'piperacillin', 'ampicillin', 
# Worked these out by splitting other sections
list3 = ['linezolid', 'Zyvox', 'Synercid', 'quinupristin', 'dalfopristin', 'quinupristin/dalfopristin', 'cephalosporin', 'ticarcillin', 'amoxicillin', 'penicillin', 'benzathine', 'piperacillin', 'ampicillin']
antibiotic_list = list1 + list2 + list3
# Drop duplicates through set
antibiotic_list = list(set(antibiotic_list))
# Drop None
antibiotic_list = [x for x in antibiotic_list if x is not None]
print(len(antibiotic_list))

101


In [ ]:
# Load all medications
medication = pd.read_csv(r"eicu-collaborative-research-database-2.0/medication.csv", dtype={'drugname': 'object'})
medication = medication[['medicationid', 'patientunitstayid', 'drugstartoffset', 'drugname', 'routeadmin', 'drugivadmixture', 'drugordercancelled', 'drugstopoffset']]
# Filter for antibiotics 
medication['flag'] = np.where(medication.drugname.str.contains('|'.join(antibiotic_list), na=False, case=False),1,0)
antibiotics = medication[medication['flag'] == 1]
antibiotics = antibiotics[antibiotics['drugordercancelled'] == 'No']
assert len(antibiotics[antibiotics['drugordercancelled'] == 'No']) == len(antibiotics)
print(antibiotics.patientunitstayid.nunique())

In [176]:
# Set those where 'drugivadmixture' == 'Yes' to 'IV' route
antibiotics.loc[antibiotics['drugivadmixture'] == 'Yes', 'routeadmin'] = 'IV'

In [177]:
# Filter for relevant delivery methods
route_list = ['IV', 'Intrav', 'PO', 'tube', 'ORAL']
antibiotics = antibiotics[antibiotics.routeadmin.str.contains('|'.join(route_list), na=False, case=False)]
routeadmin_value_counts = antibiotics.routeadmin.value_counts()

In [178]:
# Group so all IV or PO
iv_route_list = ['IV', 'Intrav']
po_route_list = ['PO', 'tube', 'ORAL']
antibiotics.loc[antibiotics.routeadmin.str.contains('|'.join(iv_route_list), na=False, case=False), 'routeadmin'] = 'IV'
antibiotics.loc[antibiotics.routeadmin.str.contains('|'.join(po_route_list), na=False, case=False), 'routeadmin'] = 'PO'

In [180]:
# Drop
antibiotics = antibiotics.drop(columns=['drugivadmixture', 'drugordercancelled', 'flag'])
antibiotics

,medicationid,patientunitstayid,drugstartoffset,drugname,routeadmin,drugstopoffset
25,7573839,141194,13422,VANCOMYCIN 1.25 GM IN NS 250 ML IVPB (REPACKAGE),IV,12622
32,10670852,141203,-129,100 ML - METRONIDAZOLE IN NACL 5-0.79 MG/ML-...,IV,1058
41,7814668,141203,1281,100 ML - METRONIDAZOLE IN NACL 5-0.79 MG/ML-...,IV,2414
42,8867081,141203,2361,VANCOMYCIN 1.25 GM IN NS 250 ML IVPB (REPACKAGE),IV,2414
55,8076824,141227,-1323,CEFEPIME HCL 2 G IJ SOLR,IV,507
...,...,...,...,...,...,...
7290293,111808256,3347198,274,PIPERACILLIN-TAZOBACTAM 3.375 G MINI-BAG PLUS,IV,80
7292726,110617275,3348568,1830,LEVOFLOXACIN,PO,4673
7295823,111068263,3350223,2431,LEVOFLOXACIN,PO,5311
7295987,111598516,3350309,2750,PIPERACILLIN-TAZOBACTAM 3.375 G MINI-BAG PLUS,IV,3726


In [205]:
# Load data
from heapq import merge

patients = pd.read_csv(r"eicu-collaborative-research-database-2.0/patient.csv")
# Select relevant columns
patients2 = patients[['patientunitstayid', 'hospitaldischargestatus', 'unitadmittime24', 'unitdischargeoffset', 'unitdischargestatus']]
# Filter for those who survived
patients2 = patients2[(patients2['hospitaldischargestatus'] == 'Alive') & (patients2['unitdischargestatus'] == 'Alive')]
patients2 = patients2.drop(columns=['hospitaldischargestatus', 'unitdischargestatus'])
# Merge
antibiotic_patients = pd.merge(antibiotics, patients2, how="left", on='patientunitstayid')


In [206]:
from datetime import datetime
from datetime import timedelta

# Convert unit admit time to day (day 0)
antibiotic_patients['unitadmittime24'] = pd.to_datetime(antibiotic_patients['unitadmittime24'])
# Create starttime and stoptime and discharge time
antibiotic_patients['drugstartoffset'] = pd.to_timedelta(antibiotic_patients['drugstartoffset'], unit='min') # Convert to timedelta
antibiotic_patients['drugstopoffset'] = pd.to_timedelta(antibiotic_patients['drugstopoffset'], unit='min') # Convert to timedelta
antibiotic_patients['unitdischargeoffset'] = pd.to_timedelta(antibiotic_patients['unitdischargeoffset'], unit='min')

antibiotic_patients['starttime'] = antibiotic_patients['unitadmittime24'] + antibiotic_patients['drugstartoffset']
antibiotic_patients['stoptime'] = antibiotic_patients['unitadmittime24'] + antibiotic_patients['drugstopoffset']
antibiotic_patients['dischargetime'] = antibiotic_patients['unitadmittime24'] + antibiotic_patients['unitdischargeoffset']

In [207]:
antibiotic_patients.groupby(['patientunitstayid']).routeadmin.nunique().value_counts()

1    45090
2     3243
Name: routeadmin, dtype: int64

In [208]:
antibiotic_patients
antibiotic_patients.patientunitstayid.nunique()
antibiotic_patients.info()

,medicationid,patientunitstayid,drugstartoffset,drugname,routeadmin,drugstopoffset,unitadmittime24,unitdischargeoffset,starttime,stoptime,dischargetime
0,7573839,141194,9 days 07:42:00,VANCOMYCIN 1.25 GM IN NS 250 ML IVPB (REPACKAGE),IV,8 days 18:22:00,2022-09-02 07:18:00,3 days 08:13:00,2022-09-11 15:00:00,2022-09-11 01:40:00,2022-09-05 15:31:00
1,10670852,141203,-1 days +21:51:00,100 ML - METRONIDAZOLE IN NACL 5-0.79 MG/ML-...,IV,0 days 17:38:00,2022-09-02 20:39:00,1 days 07:09:00,2022-09-02 18:30:00,2022-09-03 14:17:00,2022-09-04 03:48:00
2,7814668,141203,0 days 21:21:00,100 ML - METRONIDAZOLE IN NACL 5-0.79 MG/ML-...,IV,1 days 16:14:00,2022-09-02 20:39:00,1 days 07:09:00,2022-09-03 18:00:00,2022-09-04 12:53:00,2022-09-04 03:48:00
3,8867081,141203,1 days 15:21:00,VANCOMYCIN 1.25 GM IN NS 250 ML IVPB (REPACKAGE),IV,1 days 16:14:00,2022-09-02 20:39:00,1 days 07:09:00,2022-09-04 12:00:00,2022-09-04 12:53:00,2022-09-04 03:48:00
4,8076824,141227,-1 days +01:57:00,CEFEPIME HCL 2 G IJ SOLR,IV,0 days 08:27:00,2022-09-02 12:03:00,1 days 03:32:00,2022-09-01 14:00:00,2022-09-02 20:30:00,2022-09-03 15:35:00
...,...,...,...,...,...,...,...,...,...,...,...
131887,111808256,3347198,0 days 04:34:00,PIPERACILLIN-TAZOBACTAM 3.375 G MINI-BAG PLUS,IV,0 days 01:20:00,2022-09-02 05:26:00,2 days 13:08:00,2022-09-02 10:00:00,2022-09-02 06:46:00,2022-09-04 18:34:00
131888,110617275,3348568,1 days 06:30:00,LEVOFLOXACIN,PO,3 days 05:53:00,2022-09-02 13:07:00,1 days 11:44:00,2022-09-03 19:37:00,2022-09-05 19:00:00,2022-09-04 00:51:00
131889,111068263,3350223,1 days 16:31:00,LEVOFLOXACIN,PO,3 days 16:31:00,2022-09-02 20:29:00,4 days 01:02:00,2022-09-04 13:00:00,2022-09-06 13:00:00,2022-09-06 21:31:00
131890,111598516,3350309,1 days 21:50:00,PIPERACILLIN-TAZOBACTAM 3.375 G MINI-BAG PLUS,IV,2 days 14:06:00,2022-09-02 05:55:00,7 days 16:20:00,2022-09-04 03:45:00,2022-09-04 20:01:00,2022-09-09 22:15:00


48333

<class 'pandas.core.frame.DataFrame'>
Int64Index: 131892 entries, 0 to 131891
Data columns (total 11 columns):
 #   Column               Non-Null Count   Dtype          
---  ------               --------------   -----          
 0   medicationid         131892 non-null  int64          
 1   patientunitstayid    131892 non-null  int64          
 2   drugstartoffset      131892 non-null  timedelta64[ns]
 3   drugname             131892 non-null  object         
 4   routeadmin           131892 non-null  object         
 5   drugstopoffset       131892 non-null  timedelta64[ns]
 6   unitadmittime24      112060 non-null  datetime64[ns] 
 7   unitdischargeoffset  112060 non-null  timedelta64[ns]
 8   starttime            112060 non-null  datetime64[ns] 
 9   stoptime             112060 non-null  datetime64[ns] 
 10  dischargetime        112060 non-null  datetime64[ns] 
dtypes: datetime64[ns](4), int64(2), object(2), timedelta64[ns](3)
memory usage: 12.1+ MB


In [209]:
# Rename
antibiotic_patients = antibiotic_patients.rename(columns={'patientunitstayid': 'stay_id', 'routeadmin': 'route'})

In [211]:
antibiotic_patients
antibiotic_patients.stay_id.nunique()

,medicationid,stay_id,drugstartoffset,drugname,route,drugstopoffset,unitadmittime24,unitdischargeoffset,starttime,stoptime,dischargetime
0,7573839,141194,9 days 07:42:00,VANCOMYCIN 1.25 GM IN NS 250 ML IVPB (REPACKAGE),IV,8 days 18:22:00,2022-09-02 07:18:00,3 days 08:13:00,2022-09-11 15:00:00,2022-09-11 01:40:00,2022-09-05 15:31:00
1,10670852,141203,-1 days +21:51:00,100 ML - METRONIDAZOLE IN NACL 5-0.79 MG/ML-...,IV,0 days 17:38:00,2022-09-02 20:39:00,1 days 07:09:00,2022-09-02 18:30:00,2022-09-03 14:17:00,2022-09-04 03:48:00
2,7814668,141203,0 days 21:21:00,100 ML - METRONIDAZOLE IN NACL 5-0.79 MG/ML-...,IV,1 days 16:14:00,2022-09-02 20:39:00,1 days 07:09:00,2022-09-03 18:00:00,2022-09-04 12:53:00,2022-09-04 03:48:00
3,8867081,141203,1 days 15:21:00,VANCOMYCIN 1.25 GM IN NS 250 ML IVPB (REPACKAGE),IV,1 days 16:14:00,2022-09-02 20:39:00,1 days 07:09:00,2022-09-04 12:00:00,2022-09-04 12:53:00,2022-09-04 03:48:00
4,8076824,141227,-1 days +01:57:00,CEFEPIME HCL 2 G IJ SOLR,IV,0 days 08:27:00,2022-09-02 12:03:00,1 days 03:32:00,2022-09-01 14:00:00,2022-09-02 20:30:00,2022-09-03 15:35:00
...,...,...,...,...,...,...,...,...,...,...,...
131887,111808256,3347198,0 days 04:34:00,PIPERACILLIN-TAZOBACTAM 3.375 G MINI-BAG PLUS,IV,0 days 01:20:00,2022-09-02 05:26:00,2 days 13:08:00,2022-09-02 10:00:00,2022-09-02 06:46:00,2022-09-04 18:34:00
131888,110617275,3348568,1 days 06:30:00,LEVOFLOXACIN,PO,3 days 05:53:00,2022-09-02 13:07:00,1 days 11:44:00,2022-09-03 19:37:00,2022-09-05 19:00:00,2022-09-04 00:51:00
131889,111068263,3350223,1 days 16:31:00,LEVOFLOXACIN,PO,3 days 16:31:00,2022-09-02 20:29:00,4 days 01:02:00,2022-09-04 13:00:00,2022-09-06 13:00:00,2022-09-06 21:31:00
131890,111598516,3350309,1 days 21:50:00,PIPERACILLIN-TAZOBACTAM 3.375 G MINI-BAG PLUS,IV,2 days 14:06:00,2022-09-02 05:55:00,7 days 16:20:00,2022-09-04 03:45:00,2022-09-04 20:01:00,2022-09-09 22:15:00


48333

In [212]:
# Combine so one duration per icu stay and per route
antibiotic_patients2 = antibiotic_patients.groupby(['stay_id', 'route']).agg({'starttime':'min', 'stoptime':'max'})[['starttime','stoptime']].reset_index()
mask1 = antibiotic_patients2['route'].eq('IV')
mask2 = antibiotic_patients2['route'].eq('PO')
m = (
    mask1.groupby(antibiotic_patients2['stay_id']).transform('any') & 
    mask2.groupby(antibiotic_patients2['stay_id']).transform('any') & 
    antibiotic_patients2['route'].isin(['IV', 'PO'])
)
filtered_antibiotic_patients = antibiotic_patients2[m]

In [213]:
filtered_antibiotic_patients
filtered_antibiotic_patients.stay_id.nunique()
filtered_antibiotic_patients.route.value_counts()

,stay_id,route,starttime,stoptime
2,141196,IV,2022-09-03 14:00:00,2022-09-05 14:53:00
3,141196,PO,2022-09-06 12:00:00,2022-09-06 19:02:00
9,141244,IV,2022-08-24 23:15:00,2022-09-03 17:05:00
10,141244,PO,2022-08-25 19:45:00,2022-09-07 14:04:00
18,141362,IV,2022-09-02 15:00:00,2022-09-15 20:35:00
...,...,...,...,...
51443,3346897,PO,2022-09-27 17:00:00,2022-10-01 17:00:00
51458,3347124,IV,2022-09-05 16:55:00,2022-09-13 18:57:00
51459,3347124,PO,2022-08-25 06:40:00,2022-08-28 13:00:00
51557,3352613,IV,2022-08-30 01:41:00,2022-09-21 14:32:00


3243

IV    3243
PO    3243
Name: route, dtype: int64

In [214]:
# Drop thoes with NaT (died)
filtered_antibiotic_patients.dropna(inplace=True)
filtered_antibiotic_patients.stay_id.nunique()
filtered_antibiotic_patients.route.value_counts()

2942

IV    2942
PO    2942
Name: route, dtype: int64

In [215]:
# Now same as with mimic
pd.options.mode.chained_assignment = None

# Create df with one row per hosptial stay
filtered_antibiotic_patients_iv = filtered_antibiotic_patients[filtered_antibiotic_patients['route'] == 'IV']
filtered_antibiotic_patients_po = filtered_antibiotic_patients[filtered_antibiotic_patients['route'] == 'PO']
filtered_antibiotic_patients_iv.rename(columns={'starttime':'iv_starttime', 'stoptime':'iv_stoptime'}, inplace=True)
filtered_antibiotic_patients_po.rename(columns={'starttime':'po_starttime', 'stoptime':'po_stoptime'}, inplace=True)
filtered_antibiotic_patients_iv.drop(columns=['route'], inplace=True)
filtered_antibiotic_patients_po.drop(columns=['route'], inplace=True)
new_antibiotic_patients = pd.merge(filtered_antibiotic_patients_iv, filtered_antibiotic_patients_po, how="inner", on=['stay_id'])

new_antibiotic_patients.stay_id.nunique()
#new_antibiotic_patients

# Remove hours from dates
new_antibiotic_patients['iv_starttime'] = pd.to_datetime(new_antibiotic_patients['iv_starttime']).dt.date
new_antibiotic_patients['iv_stoptime'] = pd.to_datetime(new_antibiotic_patients['iv_stoptime']).dt.date
new_antibiotic_patients['po_starttime'] = pd.to_datetime(new_antibiotic_patients['po_starttime']).dt.date
new_antibiotic_patients['po_stoptime'] = pd.to_datetime(new_antibiotic_patients['po_stoptime']).dt.date
# Filter for those who have (stop) oral after IV
new_antibiotic_patients2 = new_antibiotic_patients[new_antibiotic_patients['iv_stoptime'] <= new_antibiotic_patients['po_stoptime']]

#new_antibiotic_patients2.stay_id.nunique()

# Work out duration of both and time difference between iv and oral 
new_antibiotic_patients2['iv_duration'] = new_antibiotic_patients2['iv_stoptime'] - new_antibiotic_patients2['iv_starttime']
new_antibiotic_patients2['po_duration'] = new_antibiotic_patients2['po_stoptime'] - new_antibiotic_patients2['po_starttime']
new_antibiotic_patients2['antibiotic_gap'] = new_antibiotic_patients2['po_starttime'] - new_antibiotic_patients2['iv_stoptime']
# Convert to float and unit days
new_antibiotic_patients2['iv_duration'] = new_antibiotic_patients2['iv_duration']/ np.timedelta64(1, 'D')
new_antibiotic_patients2['po_duration'] = new_antibiotic_patients2['po_duration']/ np.timedelta64(1, 'D')
new_antibiotic_patients2['antibiotic_gap'] = new_antibiotic_patients2['antibiotic_gap']/ np.timedelta64(1, 'D')

#new_antibiotic_patients2.iv_duration.plot.hist()

# Filter for relevant durations 
#new_antibiotic_patients2.stay_id.nunique()
new_antibiotic_patients3 = new_antibiotic_patients2[new_antibiotic_patients2['iv_duration'] >= 0]
#new_antibiotic_patients3.stay_id.nunique()
new_antibiotic_patients3 = new_antibiotic_patients3[new_antibiotic_patients3['po_duration'] >= 0]
#new_antibiotic_patients3.stay_id.nunique()
new_antibiotic_patients3 = new_antibiotic_patients3[new_antibiotic_patients3['iv_duration'] <= 7]
#new_antibiotic_patients3.stay_id.nunique()

#new_antibiotic_patients3.iv_duration.plot.hist()
#new_antibiotic_patients2.antibiotic_gap.plot.hist()

# Find total duration
new_antibiotic_patients3['total_duration'] = new_antibiotic_patients3['iv_duration'] + new_antibiotic_patients3['antibiotic_gap'] + new_antibiotic_patients3['po_duration']

new_antibiotic_patients3.stay_id.nunique()
#new_antibiotic_patients3

new_antibiotic_patients4 = new_antibiotic_patients3.copy()

# Create df with date range for IV and PO
IV_dates = new_antibiotic_patients4.copy()
IV_dates['date'] = IV_dates.apply(lambda x:
    pd.date_range(start=x['iv_starttime'],
                end=x['iv_stoptime'],
                #inclusive='both',
                freq='D'), axis=1)
IV_dates = IV_dates.explode('date')
IV_dates['iv_flag'] = 1

PO_dates = new_antibiotic_patients4.copy()
PO_dates['date'] = PO_dates.apply(lambda x:
    pd.date_range(start=x['po_starttime'],
                end=x['po_stoptime'],
                #inclusive='both',
                freq='D'), axis=1)
PO_dates = PO_dates.explode('date')
PO_dates['first_po_flag'] = 1
# Merge
PO_dates = PO_dates[['stay_id', 'date', 'first_po_flag']]

dates = pd.merge(IV_dates, PO_dates, on=['stay_id', 'date'], how='outer')
# Fill in NAN values
#dates['first_po_flag'] = dates['first_po_flag'].fillna(0)

# Order
dates = dates.sort_values(by=['stay_id', 'date'])
# Filter for relevant columns
dates = dates[['stay_id', 'date', 'iv_flag', 'first_po_flag']]


2942

1863

In [217]:
# Create main po_flag  - note this is changed later

dates.reset_index(inplace=True, drop=True)

dates['po_flag'] = 0

pos = -1
for x in range(len(dates)):
    pos += 1
    if dates.iloc[x]['iv_flag'] == 1:
        dates.loc[x,'po_flag'] = 0
    elif dates.iloc[x]['iv_flag'] != dates.iloc[x]['iv_flag']: # check if NaN 
        if dates.iloc[x]['first_po_flag'] == 1:
            dates.loc[x,'po_flag'] = 1
        elif dates.iloc[x]['first_po_flag'] != dates.iloc[x]['first_po_flag']: # check if NaN 
            print('???')
            dates.loc[x,'po_flag'] = 1
        
        dates.loc[x-1,'po_flag'] = 1 # Update so day IV is stopped is also positive flag for switching 


In [218]:
# iv_treatment_length - note this is changed later
cumcount = []
count = 1
pos = -1

for x in range(len(dates)):
    pos += 1
    if pos == len(dates) - 1:
        #count += 1 # add 1 to last one and append 
        cumcount.append(0) # Manually added  
    elif dates.iloc[x]['date'] == (dates.iloc[x+1]['date'] - pd.DateOffset(days=1)):
        if dates.iloc[x]['stay_id'] == dates.iloc[x+1]['stay_id']:
            if dates.iloc[x]['iv_flag'] == 1:
                cumcount.append(count)
                count += 1
            else:
                cumcount.append(0)
                count = 1
        else:
            if dates.iloc[x]['iv_flag'] == 1:
                cumcount.append(count)
            else:
                cumcount.append(0)
                count = 1
    elif dates.iloc[x]['iv_flag'] == 1:
        cumcount.append(count)
        count = 1
    else:
        cumcount.append(0)
        count = 1

#print(cumcount)
print(len(cumcount))

dates['iv_treatment_length'] = cumcount
#pd.options.display.max_rows = 100
#dates[0:100]
dates
dates.info()

15524


,stay_id,date,iv_flag,first_po_flag,po_flag,iv_treatment_length
0,141196,2022-09-03,1.0,NaN,0,1
1,141196,2022-09-04,1.0,NaN,0,2
2,141196,2022-09-05,1.0,NaN,1,3
3,141196,2022-09-06,NaN,1.0,1,0
4,141392,2022-09-02,1.0,NaN,0,1
...,...,...,...,...,...,...
15519,3346897,2022-09-27,NaN,1.0,1,0
15520,3346897,2022-09-28,NaN,1.0,1,0
15521,3346897,2022-09-29,NaN,1.0,1,0
15522,3346897,2022-09-30,NaN,1.0,1,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15524 entries, 0 to 15523
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   stay_id              15524 non-null  int64         
 1   date                 15524 non-null  datetime64[ns]
 2   iv_flag              6846 non-null   float64       
 3   first_po_flag        10839 non-null  float64       
 4   po_flag              15524 non-null  int64         
 5   iv_treatment_length  15524 non-null  int64         
dtypes: datetime64[ns](1), float64(2), int64(3)
memory usage: 727.8 KB


In [229]:
# Save
dates.to_csv('eicu_iv_switch_stayid_dates.csv', index=False)